In [ ]:
from astropy.io import fits
import numpy 
import xspec
import matplotlib.pyplot as plt
%matplotlib inline

# Visualize Lightcurve Data

In [ ]:
lc_file = 'AL1_SOLEXS_20240212/L1/SDD2/AL1_SOLEXS_20240212_SDD2_L1.lc.gz'
hdu_lc = fits.open(lc_file)
lc_data = hdu_lc[2].data
counts_lc = lc_data['COUNTS_ALL']

In [ ]:
fig, ax = plt.subplots()

fig.set_size_inches(16,9)

ax.semilogy(lc_data['TIME'],counts_lc,label='1 second')
ax.grid()

## Flare Peak Time Interval

In [ ]:
fig, ax = plt.subplots()

fig.set_size_inches(16,9)

ax.semilogy(lc_data['TIME'],counts_lc,label='1 second')
ax.axvline(lc_data['TIME'][13605],color='k',ls='--')
ax.axvline(lc_data['TIME'][13635],color='k',ls='--')
ax.grid()

ax.set_xlim([lc_data['TIME'][13000],lc_data['TIME'][15000]])

## Generating SoLEXS Spectrum Corresponding to Flare Peak

In [ ]:
!solexs-genspec -i AL1_SOLEXS_20240212/L1/SDD2/AL1_SOLEXS_20240212_SDD2_L1.pi.gz -tstart 1707689805 -tstop 1707689835 -gti AL1_SOLEXS_20240212/L1/SDD2/AL1_SOLEXS_20240212_SDD2_L1.gti.gz

# Spectral Fitting (Single Temperature)

In [ ]:
xspec.AllModels.lmod('chspec',dirPath='/path/to/chspec/')

In [ ]:
xspec.AllData.clear()
xspec.AllModels.clear()

In [ ]:
specfile = 'AL1_SOLEXS_20240212_SDD2_L1_034645_034715.pi'

In [ ]:
spec = xspec.Spectrum(specfile)

In [ ]:
xspec.AllModels.systematic = 0.04 # set systematic error to be 4%

In [ ]:
xspec.Plot.device = '/xw'
xspec.Plot.xAxis = 'keV'

In [ ]:
spec.ignore("**-2.8 12.0-**")  # Fit energy range 2.8 to 12 keV

In [ ]:
model = xspec.Model("chisoth")

In [ ]:
model(1).values=[7] #Temperature initial guess

#Thaw adundances
model(18).frozen = False # Ar Abundance
model(20).frozen = False # Ca Abundance
model(26).frozen = False # Fe Abundance
model(28).frozen = False # Ni Abundance

In [ ]:
xspec.Fit.perform()

In [ ]:
chisq = xspec.Fit.statistic # Total chi-square value
dof = xspec.Fit.dof # Degrees of freedom
red_chi = chisq / dof # Reduced chi-square
print(f'Reduced chi square = {red_chi}')

In [ ]:
xspec.Plot("ldata delchi")

# Spectral Fitting (Two Temperature)

In [ ]:
xspec.AllModels.lmod('chspec',dirPath='/path/to/chspec/')

In [ ]:
xspec.AllData.clear()
xspec.AllModels.clear()

In [ ]:
specfile = 'AL1_SOLEXS_20240212_SDD2_L1_034645_034715.pi'

In [ ]:
spec = xspec.Spectrum(specfile)

In [ ]:
xspec.AllModels.systematic = 0.04 # set systematic error to be 4%

In [ ]:
xspec.Plot.device = '/xw'
xspec.Plot.xAxis = 'keV'

In [ ]:
spec.ignore("**-2.0 12.0-**")  # Fit energy range 2.8 to 12 keV

In [ ]:
model = xspec.Model("chisoth + chisoth")

In [ ]:
model(1).values=[7.3] #First Temperature initial guess

#Thaw adundances
model(18).frozen = False # Ar Abundance
model(20).frozen = False # Ca Abundance
model(26).frozen = False # Fe Abundance
model(28).frozen = False # Ni Abundance

model(32).values=[7] #Second Temperature initial guess

In [ ]:
xspec.Fit.perform()

In [ ]:
chisq = xspec.Fit.statistic # Total chi-square value
dof = xspec.Fit.dof # Degrees of freedom
red_chi = chisq / dof # Reduced chi-square
print(f'Reduced chi square = {red_chi}')

In [ ]:
xspec.Plot("ldata delchi")